# Fudan PRML 23Spring Assignment2


<font color='red'>*Your Name, Student ID and Date: [Name], [Student ID], [Date]*</font>

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission.*


<font color='red' size = 3> *DDL: 2023.06.11 23:59* </font>

## 1. FDUNN: your toy torch-like deep learning library (60 points)

In this assignment, you will fist implement your own torch-like deep learning library with `numpy`, named `fdunn`.

PyTorch: [Link](https://pytorch.org/)

In [38]:
# setup code, auto reload your .py file
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
# numpy
import numpy as np
np.random.seed(233)

In [40]:
# matplotlib
%matplotlib inline
from matplotlib import pyplot as plt

You have to impletement several standard deep neural modelwork modules in the `./fdunn` folder:
1.   linear/conv/pooling
2.   activation
3.   loss
4.   optim
5.   trainer

We have written most of the code for you already, and you only need to fill in the most essential parts. We have also prepared several test cases for you to check if your code works correctly.

Furthermore, you can also test the accuracy of your code by comparing its output with the output of sk-learn.

In [41]:
import sys
sys.path.append(r'C:\Users\Life_Dancer\Desktop\PRML-Spring23-FDU\assignment2\fdunn')

from fdunn import *

### 1. Get dataset MNIST

In [42]:
from fdunn.data.data_loader import get_dataset

dataset = 'MNIST'
trainloader, testloader = get_dataset(dataset)

### 2. FNN

In [43]:
from fdunn.modules.linear import Linear
from fdunn.modules.activation import Sigmoid
from fdunn.modules.loss import BCELoss, CrossEntropyLoss
from fdunn.optim.sgd import SGD

class FNN:
    def __init__(self, in_features, hidden_sizes, num_classes):
        self.linear1 = Linear(in_features, hidden_sizes[0])
        self.sigmoid = Sigmoid()
        self.hidden_layers = []
        for i in range(len(hidden_sizes) - 1):
            layer = Linear(hidden_sizes[i], hidden_sizes[i+1])
            self.hidden_layers.append(layer)
        self.linear_final = Linear(hidden_sizes[-1], num_classes)
        self.layers = [self.linear1, self.sigmoid] + self.hidden_layers + [self.linear_final]
    
    def forward(self, input):
        output = self.linear1.forward(input)
        output = self.sigmoid.forward(output)
        for layer in self.hidden_layers:
            output = layer.forward(output)
            output = self.sigmoid.forward(output)
        output = self.linear_final.forward(output)
        return output
    
    def backward(self, output_grad):
        output_grad = self.linear_final.backward(output_grad)
        for layer in reversed(self.hidden_layers):
            output_grad = self.sigmoid.backward(output_grad)
            output_grad = layer.backward(output_grad)
        output_grad = self.sigmoid.backward(output_grad)
        output_grad = self.linear1.backward(output_grad)
        return output_grad

In [44]:
in_features = 784
hidden_sizes = [64, 64] 
num_classes = 10

model = FNN(in_features, hidden_sizes, num_classes)
criterion = BCELoss(model)

lr = 1e-7
optimizer = SGD(model, lr)

init: W.shape=(64, 784), b.shape=(64,)
init: W.shape=(64, 64), b.shape=(64,)
init: W.shape=(10, 64), b.shape=(10,)


In [45]:
import torch.nn.functional as F

def epoch(mode, dataloader, model, optimizer, criterion):
    loss_avg, acc_avg, num_exp = 0, 0, 0

    for i_batch, datum in enumerate(dataloader):
        img = datum[0]
        lab = datum[1]

        n_b = lab.shape[0]
        truth = lab

        output = model.forward(img)

        lab = F.one_hot(lab, 10)
        lab = lab.numpy()

        loss = criterion.forward(output, lab)
        predicted = np.argmax(output, 1)

        correct = (predicted == truth.numpy()).sum()

        loss_avg += loss.item()*n_b
        acc_avg += correct.item()
        num_exp += n_b

        if mode == 'train':
            criterion.backward()
            optimizer.step()
            
        # print(f"Batch: {i_batch}, Loss: {loss.item()}, Accuracy: {correct.item() / n_b}")

    loss_avg /= num_exp
    acc_avg /= num_exp

    return loss_avg, acc_avg

In [46]:
train_epochs = 20
lr_schedule = [10, 15]
decay = True

for e in range(train_epochs):

    train_loss, train_acc = epoch("train", dataloader=trainloader, model=model, optimizer=optimizer, criterion=criterion)

    test_loss, test_acc = epoch("test", dataloader=testloader, model=model, optimizer=None, criterion=criterion)

    print("Epoch: {}\tReal Acc: {}\tTest Acc: {}".format(e, train_acc, test_acc))

    if e in lr_schedule and decay:
        lr *= 0.1
        optimizer = SGD(model, lr=lr)


Epoch: 0	Real Acc: 0.11236666666666667	Test Acc: 0.1135
Epoch: 1	Real Acc: 0.11236666666666667	Test Acc: 0.1135
Epoch: 2	Real Acc: 0.11236666666666667	Test Acc: 0.1136
Epoch: 3	Real Acc: 0.11275	Test Acc: 0.1141
Epoch: 4	Real Acc: 0.11473333333333334	Test Acc: 0.116
Epoch: 5	Real Acc: 0.11715	Test Acc: 0.1183
Epoch: 6	Real Acc: 0.11923333333333333	Test Acc: 0.1201
Epoch: 7	Real Acc: 0.1207	Test Acc: 0.1206
Epoch: 8	Real Acc: 0.12156666666666667	Test Acc: 0.1206
Epoch: 9	Real Acc: 0.1219	Test Acc: 0.1212
Epoch: 10	Real Acc: 0.1219	Test Acc: 0.1221
Epoch: 11	Real Acc: 0.12358333333333334	Test Acc: 0.1221
Epoch: 12	Real Acc: 0.12346666666666667	Test Acc: 0.122
Epoch: 13	Real Acc: 0.12353333333333333	Test Acc: 0.122
Epoch: 14	Real Acc: 0.12346666666666667	Test Acc: 0.122
Epoch: 15	Real Acc: 0.1234	Test Acc: 0.122
Epoch: 16	Real Acc: 0.12355	Test Acc: 0.122
Epoch: 17	Real Acc: 0.12355	Test Acc: 0.1219
Epoch: 18	Real Acc: 0.1235	Test Acc: 0.1219
Epoch: 19	Real Acc: 0.12353333333333333	Test A

### CNN

In [112]:
from fdunn.modules.linear import Linear
from fdunn.modules.activation import Sigmoid
from fdunn.modules.loss import BCELoss, CrossEntropyLoss
from fdunn.optim.sgd import SGD
from fdunn.modules.conv import Conv2d
from fdunn.modules.pooling import MaxPool2d

class CNN:
    def __init__(self, in_channels, out_channels, kernel_size, stride_size, hidden_sizes, num_classes):
        self.conv = Conv2d(in_channels, out_channels, kernel_size)
        self.sigmoid = Sigmoid()
        self.pool = MaxPool2d(kernel_size, stride_size)
        self.hidden_layers = []
        for i in range(1, len(hidden_sizes)):
            layer = Linear(hidden_sizes[i-1], hidden_sizes[i])
            self.hidden_layers.append(layer)
        self.linear_final = Linear(hidden_sizes[-1], num_classes)
        self.layers = [self.conv, self.sigmoid] + self.hidden_layers + [self.linear_final]
    
    def forward(self, input):
        output = self.conv.forward(input)
        output = self.sigmoid.forward(output)
        output = self.pool.forward(output)
        # print('output.shape: ', output.shape)  # 将特征图展平为向量
        for layer in self.hidden_layers:
            output = layer.forward(output)
            output = self.sigmoid.forward(output)
        output = self.linear_final.forward(output)
        return output
    
    def backward(self, output_grad):
        output_grad = self.linear_final.backward(output_grad)
        for layer in reversed(self.hidden_layers):
            output_grad = layer.backward(output_grad)
            output_grad = self.sigmoid.backward(output_grad)
        output_grad = self.pool.backward(output_grad)
        output_grad = self.sigmoid.backward(output_grad)
        output_grad = self.conv.backward(output_grad)
        return output_grad

In [113]:
in_channels = 1
out_channels = 4
kernel_size = 3
stride_size = 2
hidden_sizes = [576]  # 两个隐藏层，大小逐渐减少
num_classes = 10

model = CNN(in_channels, out_channels, kernel_size, stride_size, hidden_sizes, num_classes)
criterion = BCELoss(model)

lr = 1e-7
optimizer = SGD(model, lr)

init: W.shape=(10, 576), b.shape=(10,)


In [114]:
import torch.nn.functional as F

def epoch(mode, dataloader, model, optimizer, criterion):
    loss_avg, acc_avg, num_exp = 0, 0, 0

    for i_batch, datum in enumerate(dataloader):
        img = datum[0]
        lab = datum[1]

        n_b = lab.shape[0]
        truth = lab

        output = model.forward(img)

        lab = F.one_hot(lab, 10)
        lab = lab.numpy()

        loss = criterion.forward(output, lab)
        predicted = np.argmax(output, 1)

        correct = (predicted == truth.numpy()).sum()

        loss_avg += loss.item()*n_b
        acc_avg += correct.item()
        num_exp += n_b

        if mode == 'train':
            criterion.backward()
            optimizer.step()
            
        print(f"Batch: {i_batch}, Loss: {loss.item()}, Accuracy: {correct.item() / n_b}")

    loss_avg /= num_exp
    acc_avg /= num_exp

    return loss_avg, acc_avg

In [115]:
train_epochs = 20
lr_schedule = [10, 15]
decay = True

for e in range(train_epochs):

    train_loss, train_acc = epoch("train", dataloader=trainloader, model=model, optimizer=optimizer, criterion=criterion)

    test_loss, test_acc = epoch("test", dataloader=testloader, model=model, optimizer=None, criterion=criterion)

    print("Epoch: {}\tReal Acc: {}\tTest Acc: {}".format(e, train_acc, test_acc))

    if e in lr_schedule and decay:
        lr *= 0.1
        optimizer = SGD(model, lr=lr)


Batch: 0, Loss: 0.7212353715629498, Accuracy: 0.125
Batch: 1, Loss: 0.6967853556378223, Accuracy: 0.1875
Batch: 2, Loss: 0.7047886567656144, Accuracy: 0.15625
Batch: 3, Loss: 0.7049310861545516, Accuracy: 0.140625
Batch: 4, Loss: 0.7040385231962347, Accuracy: 0.171875
Batch: 5, Loss: 0.6850737349302332, Accuracy: 0.203125
Batch: 6, Loss: 0.6871274885770369, Accuracy: 0.1328125
Batch: 7, Loss: 0.6787056250731768, Accuracy: 0.1875
Batch: 8, Loss: 0.6878268940277004, Accuracy: 0.1328125
Batch: 9, Loss: 0.6694310932888916, Accuracy: 0.2109375
Batch: 10, Loss: 0.6733723068972125, Accuracy: 0.171875
Batch: 11, Loss: 0.66884048137513, Accuracy: 0.2109375
Batch: 12, Loss: 0.6697977603948073, Accuracy: 0.140625
Batch: 13, Loss: 0.675627156495503, Accuracy: 0.1796875
Batch: 14, Loss: 0.6727670802753571, Accuracy: 0.1640625
Batch: 15, Loss: 0.675291125915819, Accuracy: 0.1171875
Batch: 16, Loss: 0.6704639261693701, Accuracy: 0.140625
Batch: 17, Loss: 0.6671598816800476, Accuracy: 0.1640625
Batch:

KeyboardInterrupt: 